## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [1]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        self.codigo = codigo
        self.nombre = nombre
        self.tipo = tipo
        self._precio = None
        self._cantidad = None
        self.precio = precio
        self.cantidad = cantidad

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, value):
        if 10 <= value <= 10000:
            self._precio = value

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, value):
        if 0 <= value <= 1000:
            self._cantidad = value

    def valorTotal(self):
        return self.precio * self.cantidad

    def __repr__(self):
        return (f"Producto(codigo={self.codigo}, nombre={self.nombre}, "
                f"precio={self.precio}, tipo={self.tipo}, cantidad={self.cantidad})")

In [ ]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [3]:
class Oferta:
    def __init__(self, descripcion, codigos=None, tipos=None):
        if not (1 <= len(descripcion) <= 100):
            raise ValueError("La descripción debe tener entre 1 y 100 caracteres.")
        self.descripcion = descripcion
        self.codigos = codigos or []
        self.tipos = tipos or []

    def es_aplicable(self, producto):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

    def aplicar(self, producto, cantidad):
        raise NotImplementedError("Este método debe ser implementado por las subclases.")

class OfertaDescuento(Oferta):
    def __init__(self, descuento, codigos=None, tipos=None):
        super().__init__(f"Descuento {descuento}%", codigos, tipos)
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.es_aplicable(producto):
            return (producto.precio * cantidad) * (self.descuento / 100)
        return 0

class Oferta2x1(Oferta):
    def __init__(self, codigos=None, tipos=None):
        super().__init__("Oferta 2x1", codigos, tipos)

    def calcularDescuento(self, producto, cantidad):
        if self.es_aplicable(producto) and cantidad >= 2:
            return (cantidad // 2) * producto.precio
        return 0

In [ ]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [5]:
class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    @staticmethod
    def leer(archivo):
        catalogo = Catalogo()
        with open(archivo, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            headers = lines[0].strip().split(',')
            for line in lines[1:]:
                row = line.strip().split(',')
                producto = Producto(
                    codigo=row[headers.index('codigo')],
                    nombre=row[headers.index('nombre')],
                    precio=float(row[headers.index('precio')]),
                    tipo=row[headers.index('tipo')],
                    cantidad=int(row[headers.index('cantidad')])
                )
                catalogo.agregar(producto)
        return catalogo

    def guardar(self, archivo):
        with open(archivo, 'w', encoding='utf-8') as file:
            headers = ['codigo', 'nombre', 'precio', 'tipo', 'cantidad']
            file.write(','.join(headers) + '\n')
            for producto in self.productos:
                row = [
                    producto.codigo,
                    producto.nombre,
                    str(producto.precio),
                    producto.tipo,
                    str(producto.cantidad)
                ]
                file.write(','.join(row) + '\n')

    def agregar(self, *productos):
        self.productos.extend(productos)

    def buscar(self, codigo):
        return next((producto for producto in self.productos if producto.codigo == codigo), None)

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscar_oferta(self, producto):
        return next((oferta for oferta in self.ofertas if oferta.es_aplicable(producto)), None)

    def calcularDescuento(self, producto, cantidad):
        oferta = self.buscar_oferta(producto)
        return oferta.calcularDescuento(producto, cantidad) if oferta else 0

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos)

    @property
    def valorTotal(self):
        return sum(producto.valorTotal() for producto in self.productos)

    def vender(self, producto, cantidad):
        if producto.cantidad >= cantidad:
            producto.cantidad -= cantidad
        else:
            raise ValueError("Cantidad insuficiente en el inventario")

    def informe(self):
        informe = [
            "INFORME CATALOGO",
            f"Cantidad de productos:   {self.cantidadProductos}",
            f"Cantidad de unidades:    {self.cantidadUnidades}",
            f"Precio Promedio:       $ {self.valorTotal / self.cantidadUnidades:.2f}" if self.cantidadUnidades > 0 else "Precio Promedio:       $ 0.00",
            f"Valor total:           $ {self.valorTotal:.2f}",
            "Tipos de productos: "
        ]

        tipos = {}
        for producto in self.productos:
            if producto.tipo not in tipos:
                tipos[producto.tipo] = {'unidades': 0, 'total_precio': 0}
            tipos[producto.tipo]['unidades'] += producto.cantidad
            tipos[producto.tipo]['total_precio'] += producto.precio * producto.cantidad

        for tipo, datos in tipos.items():
            if datos['unidades'] > 0:
                precio_promedio = datos['total_precio'] / datos['unidades']
                informe.append(f"  - {tipo}              :  {datos['unidades']}u x $ {precio_promedio:.2f}")
            else:
                informe.append(f"  - {tipo}              :  0u")

        informe.append("Ofertas:")
        for oferta in self.ofertas:
            informe.append(f"  - {oferta.descripcion}")

        return "\n".join(informe)


In [ ]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [7]:
class Cliente:
    def __init__(self, nombre, cuit):
        self.nombre = nombre
        self.cuit = cuit

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
        if 1 <= len(value) <= 100:
            self._nombre = value

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        if self._validar_cuit(value):
            self._cuit = value

    def _validar_cuit(self, value):
        partes = value.split('-')
        return (
            len(partes) == 3 and
            len(partes[0]) == 2 and
            len(partes[1]) == 8 and
            len(partes[2]) == 1 and
            all(parte.isdigit() for parte in partes)
        )

In [ ]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

In [9]:
from datetime import datetime

class Factura:
    numero_factura = 100

    @classmethod
    def ultimaFactura(cls, numero):
        cls.numero_factura = numero

    @classmethod
    def nuevoNumero(cls):
        cls.numero_factura += 1
        return cls.numero_factura

    def __init__(self, catalogo, cliente):
        self.catalogo = catalogo
        self.cliente = cliente
        self.productos = []
        self.numero = Factura.nuevoNumero()
        self.fecha = datetime.now().strftime('%Y-%m-%d')
        self._subtotal = 0
        self._descuentos = 0

    def agregar(self, producto, cantidad):
        if cantidad <= 0:
            raise ValueError("La cantidad debe ser mayor a cero")

        self.catalogo.vender(producto, cantidad)

        for i, (prod, qty) in enumerate(self.productos):
            if prod == producto:
                self.productos[i] = (prod, qty + cantidad)
                break
        else:
            self.productos.append((producto, cantidad))

        self.calcular_totales()

    def calcular_totales(self):
        self._subtotal = sum(producto.precio * cantidad for producto, cantidad in self.productos)
        self._descuentos = sum(self.catalogo.calcularDescuento(producto, cantidad) for producto, cantidad in self.productos)

    @property
    def subtotal(self):
        return self._subtotal

    @property
    def descuentos(self):
        return self._descuentos

    @property
    def total(self):
        return self.subtotal - self.descuentos

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(cantidad for _, cantidad in self.productos)

    def imprimir(self):
        impresion = [
            f"Factura: {self.numero}",
            f"Fecha  : {self.fecha}",
            f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})",
            ""
        ]

        for producto, cantidad in self.productos:
            subtotal_producto = producto.precio * cantidad
            impresion.append(f"- {cantidad} {producto.nombre:<20} x ${producto.precio:.2f} = ${subtotal_producto:.2f}")

            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            if descuento > 0:
                if any(isinstance(oferta, Oferta2x1) and producto.tipo in oferta.tipos for oferta in self.catalogo.ofertas):
                    descripcion_oferta = "Oferta 2x1"
                    impresion.append(f"      {descripcion_oferta:<40} - ${descuento:.2f}")
                else:
                    for oferta in self.catalogo.ofertas:
                        if isinstance(oferta, OfertaDescuento) and producto.codigo in oferta.codigos:
                            descripcion_oferta = f"Descuento {oferta.descuento}%"
                            impresion.append(f"      {descripcion_oferta:<40} - ${descuento:.2f}")
                            break

        impresion.append(f"{'Subtotal:':>50}   ${self.subtotal:.2f}")
        impresion.append(f"{'Descuentos:':>50} - ${self.descuentos:.2f}")
        impresion.append(f"{'-' * 50}")
        impresion.append(f"TOTAL:   ${self.total:.2f}")

        return "\n".join(impresion)

In [ ]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

In [ ]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")